In [ ]:
import glob
import numpy as np
from scipy import signal
import matplotlib.pyplot as plt
import os
from tqdm import tqdm
import plotly.express as px
import pandas as pd
import plotly.io as pio
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from plotly.express.colors import sample_colorscale
from datetime import datetime, timedelta
import matplotlib as mpl
import matplotlib.animation as animation
import simplekml
import yaml
from scipy import constants
from scipy.optimize import minimize
from tqdm.contrib.concurrent import process_map

from utils.antenna import *
from utils.sig_processor import *   
from utils.vhcl_processor import * 
from utils.config_parser import *
from post_process import *

SIGMF = True   # Set to True if processing sigmf files, False for npz files

if SIGMF:
    # If you are post-processing data from sigmf files
    EXP_DATE = "2023-12-15"
    EXP_DIR = glob.glob(f"../A2G_Channel_Measurements/{EXP_DATE}*")
else:
    # Use this if you are post-processing data from npz files
    EXP_DATE = "2023-12-15"
    EXP_DIR = glob.glob(f"../field_data/measurements/{EXP_DATE}*")
print(f"Processing date: {EXP_DATE}")

## Parse this information from config or sigmf-meta file
sample_rate = 56e6
OFDM_LEN = 450
ZC_LEN = 401
OFDM_GUARD = 300

CC1 = (35.773851, -78.677010)
LW1 = (35.72747884, -78.69591754) 
LW2 = (35.72821230,-78.70093459)
H_TOWER_LW1 = 12
H_TOWER_LW2 = 10
UTC_TIME_OFF = 3
UTC_OFF_REQ = True

%matplotlib inline

In [ ]:
processor = PostProcessor() 

FORCE = False
SIGMF = True  # Set to True if processing sigmf files, False for npz files
# processed = processor.process_dates(EXP_DIR, FORCE, SIGMF)
## If one specific date is to be processed 
processed = processor.process_date(EXP_DIR[0], FORCE, SIGMF)
opac_colors, fl_colors = processor.get_colorscale()

In [ ]:
processor.plot_time_vs_power(0)

In [ ]:
processor.plot_time_vs_loc(0)

In [ ]:
processor.plot_freqoff_vs_time(0)

In [ ]:
processor.plot_multiple_freqoff_vs_time()

In [ ]:
processor.plot_snr_vs_time(0)

In [ ]:
processor.plot_dist_vs_snr(1)

In [ ]:
processor.plot_snr_vs_loc(0)

# Plots

In [ ]:
fig = go.Figure()

alpha = 0.2

for i in range(len(processed["meas"])):
    if processed["meas"][i]["alt"].max() < 40:  # 30 Meter plan
        if len(processed["meas"][i]) < 100:
            continue
        processed["meas"][i] = processed["meas"][i].sort_values(by="time")
        fig.add_trace(go.Scatter(x=processed["meas"][i]["dist"], y=processed["meas"][i]["avgPower"], mode='lines+markers', marker={
                      "color": opac_colors[i]}, name=f"{processed['freq'][i]} MHz {processed['waveType'][i]}"))
        avg_power_ewma = processed["meas"][i]["avgPower"].ewm(alpha=0.2).mean()
        fig.add_trace(go.Scatter(x=processed["meas"][i]["dist"], y=avg_power_ewma, mode='lines', marker={
                      "color": fl_colors[i]}, name=f"{processed['freq'][i]} MHz {processed['waveType'][i]} EWMA alpha={alpha} "))


fig.update_layout(width=1920, height=1080, title="Average Power vs Distance", xaxis_title="Distance (m)", yaxis_title="Average Power (dBm)", legend=dict(
    yanchor="bottom", y=0.01, xanchor="left", x=0.01),
    font=dict(family="Courier New, monospace", size=32),
    legend_title_text="Frequency (MHz) Waveform")

fig.show()

In [ ]:
fig = go.Figure()
altitudes = np.array([30, 60, 90])
for i in range(len(processed["meas"])):
    if len(processed["meas"][i]) < 100:
        continue
    processed["meas"][i] = processed["meas"][i].sort_values(by="time")
    alt = np.floor(processed["meas"][i]["alt"].max())
    fig.add_trace(go.Scatter(x=processed["meas"][i]["dist"], y=processed["meas"][i]["avgPower"], mode='lines+markers', marker={
                    "color": opac_colors[i*4]}, name=f"{processed['freq'][i]} MHz Altitude = {alt} m"))

    avg_power_ewma = processed["meas"][i]["avgPower"].ewm(alpha=0.2).mean()
    fig.add_trace(go.Scatter(x=processed["meas"][i]["dist"], y=avg_power_ewma, mode='lines', marker={
                    "color": fl_colors[i*4]}, name=f"{processed['freq'][i]} MHz EWMA <br>Alpha={alpha} Altitude = {alt} m"))
    
fig.update_layout(width=1920, height=1080,  xaxis_title="Distance (m)", yaxis_title="Average Power (dBm)", legend=dict(
    yanchor="top", y=0.92, xanchor="left", x=1.01),
    font=dict(family="Courier New, monospace", size=20))

fig.show()

In [ ]:
df = processed["meas"][0]
freq = processed["freq"][0]
waveType = processed["waveType"][0]

In [ ]:
# Don't share the token with others
px.set_mapbox_access_token(
    "pk.eyJ1Ijoic2VnbWVudGZhdWx0IiwiYSI6ImNsYTh5bTU5aDA2OWQzb3F4c29rajR4MjAifQ.XyOdrlhpVOUJZ18YFNwd8A"
)

fig =px.scatter_mapbox(
        lat=df.lat,
        lon=df.lon,
        color=df.avgPower,
        labels={"avgPower": "Power(dBm)"},
        title=f"Power vs Location for Center Freq. = {freq} MHz and {waveType} waveform",
        mapbox_style="satellite",
        color_continuous_scale=px.colors.sequential.thermal,
        size_max=11,
        size=df["lat"],
        zoom=16.2,
        width=1400,
        height=1000
    )

fig.update_coloraxes(showscale=True)
fig.show()

In [ ]:
fig = go.Figure(
    data=[
        go.Scatter3d(
            x=df.lat,
            y=df.lon,
            z=df.alt,
            mode="markers",
            marker=dict(
                size=12,
                color=df.avgPower,  # set color to an array/list of desired values
                colorscale="Thermal",  # choose a colorscale
                opacity=0.8,
            ),
        )
    ]
)

# tight layout
fig.update_layout(margin=dict(l=0, r=0, b=0, t=0))
fig.show()

In [ ]:
# 3D Plot of Power for KML

RESULT_DIR = processed["resultDir"][0]
inferno = mpl.colormaps['jet'].resampled(320)
kml = simplekml.Kml()

prev_loc = None 
for row in df.iterrows():
    ls = kml.newlinestring()
    ls.altitudemode = simplekml.AltitudeMode.relativetoground
    ls.style.linestyle.width = 10
    rgb_r = inferno((row[1]["avgPower"]- df["avgPower"].min()) / (df["avgPower"].max() - df["avgPower"].min())) 
    ls.style.linestyle.color = simplekml.Color.rgb(int(rgb_r[0]*255), int(rgb_r[1]*255), int(rgb_r[2]*255))
    if not prev_loc:
        ls.coords = [(row[1]["lon"], row[1]["lat"], row[1]["alt"])] 
    else: 
        ls.coords = [(row[1]["lon"], row[1]["lat"], row[1]["alt"]), prev_loc] 
    prev_loc = (row[1]["lon"], row[1]["lat"], row[1]["alt"])

kml.save(RESULT_DIR + "Power_GPS3D.kml" )

In [ ]:
# Color map for KML
inferno = mpl.colormaps['jet'].resampled(320)

In [ ]:
# 3D Plot of Path Loss
RESULT_DIR = processed["resultDir"][0]
viridis = mpl.colormaps['viridis_r'].resampled(24)
kml = simplekml.Kml()
maxPL = df["avg_pl"].argmax() 
prev_loc = None
for row in df.iterrows():
    ls = kml.newlinestring()
    ls.altitudemode = simplekml.AltitudeMode.relativetoground
    ls.style.linestyle.width = 10
    rgb_r = viridis(row[1]["avg_pl"]/maxPL) 
    ls.style.linestyle.color = simplekml.Color.rgb(int(rgb_r[0]*255), int(rgb_r[1]*255), int(rgb_r[2]*255))
    if not prev_loc:
        ls.coords = [(row[1]["lon"], row[1]["lat"], row[1]["alt"])] 
    else: 
        ls.coords = [(row[1]["lon"], row[1]["lat"], row[1]["alt"]), prev_loc] 
    prev_loc = (row[1]["lon"], row[1]["lat"], row[1]["alt"])

kml.save(RESULT_DIR + "PathLoss_GPS3D.kml")

In [ ]:
px.set_mapbox_access_token(
    "pk.eyJ1Ijoic2VnbWVudGZhdWx0IiwiYSI6ImNsYTh5bTU5aDA2OWQzb3F4c29rajR4MjAifQ.XyOdrlhpVOUJZ18YFNwd8A"
)
RESULT_DIR = processed["resultDir"][0]

fig = px.scatter_mapbox(
    df,
    lat="lat",
    lon="lon",
    color="avg_pl",
    labels={"avg_pl": "Path Loss(dB)"},
    # title="Power Distribution Between CC1 and PN",
    mapbox_style="satellite",
    color_continuous_scale=px.colors.sequential.thermal,
    size_max=11,
    size="lat", # Just for making the markers bigger
    zoom=16.2,
    width=1400,
    height=1000
)

fig.update_coloraxes(showscale=True)
fig.show()

In [ ]:
fig = go.Figure()

fig.add_trace(
    go.Scatter(x=df.time, y=df.dist, mode="markers", name="GPS Based Distance")
)

fig.add_trace(
    go.Scatter(
        x=df.time,
        y=df.est_dist,
        mode="markers",
        name="Estimated Distance from the Received Signal",
    )
)

fig.update_layout(
    autosize=False,
    width=2400,
    height=1000,
    margin=dict(l=50, r=50, b=100, t=100, pad=4),
    title_x=0.5,
    font=dict(family="Courier New, monospace", size=36),
    xaxis_title="Time (s)",
    yaxis_title="Distance (m)",
)

fig.update_traces(marker=dict(size=16), selector=dict(mode="markers"))
fig.update_layout(
    legend=dict(
        yanchor="top",
        y=1.15,
        xanchor="left",
        font=dict(family="Courier New, monospace", size=36, color="black"),
        x=0.01,
        bgcolor="silver",
    ),
    plot_bgcolor="white",
)

fig.update_xaxes(showline=True, linewidth=2, linecolor="black")
fig.update_yaxes(showline=True, linewidth=3, linecolor="black")

fig.show()

In [ ]:
fig = px.scatter(df, x="dist", y="avg_pl", color="avgPower", labels={"avg_pl": "Path Loss (dB)", "dist":"Distance (m)", "time": "Time (s)"}, width=2000, height=1000, title="Distance vs Path Loss")
fig.show()

# Path Loss

In [ ]:
current_meas = 1 
df = processed["meas"][current_meas-1]
cfg = processed["config"][current_meas-1]
freq = cfg.USRP_CONF.CENTER_FREQ / 1e6
waveType = cfg.WAVEFORM
RESULT_DIR = processed["resultDir"][current_meas-1]
RES_DATE_DIR = "../field_data/general-results/" + RESULT_DIR.split("/")[-2].split("_")[0] + "/"
if not os.path.exists(RES_DATE_DIR):
    os.makedirs(RES_DATE_DIR)

In [ ]:
def calc_pathloss(df, cfg, res_dir):
    freq = cfg.USRP_CONF.CENTER_FREQ / 1e6
    
    df = df.sort_values("time")
    dist = df["dist"].values
    tx_ant_gain = processor.tx_ant.getGain(df.aod_theta-1.7, df.aod_phi)
    rx_ant_gain = processor.rx_ant.getGain(df.aoa_theta+1.9+df.yaw, df.aoa_phi-.07+df.pitch)

    lmbd = 3e8 / (freq*1e6) 

    pl_free = 10*np.log10((4*np.pi*dist)/(lmbd)) - (tx_ant_gain + rx_ant_gain)

    d_los = dist 
    d_ref = np.sqrt((df.alt.values + H_TOWER_LW1) ** 2 + df.h_dist.values ** 2) 
    phi = 2* np.pi * (d_ref - d_los) / lmbd 
    G_LOS = 1 
    G_REF = 0.3 
    REF_COEFF = 0.4
    los_coef = np.sqrt(G_LOS) / d_los 
    ref_coef = np.sqrt(G_REF) / d_ref * np.exp(-1j*phi) * REF_COEFF
    pl_2ray = -20*np.log10(lmbd * np.abs(los_coef + ref_coef)/(4*np.pi)) - (tx_ant_gain + rx_ant_gain)
    
    def mse(initial_params):
        aod_theta_off, aoa_theta_off, initial_power_off = initial_params
        tx_ant_gain = processor.tx_ant.getGain(df.aod_theta+aod_theta_off, df.aod_phi)
        rx_ant_gain = processor.rx_ant.getGain(df.aoa_theta+df.yaw+aoa_theta_off, df.aoa_phi+df.pitch)
        
        pl_2ray = -20*np.log10(lmbd * np.abs(los_coef + ref_coef)/(4*np.pi)) - (tx_ant_gain + rx_ant_gain)
        
        return np.mean((df["avg_pl"] - (pl_2ray + initial_power_off))**2)


    def rmse(initial_params):
        return np.sqrt(mse(initial_params))
    
    initial_params = [-3,3, 15]
    bounds = [(-3,3), (-3,3), (-30, 30)]

    try:
        res = minimize(rmse, initial_params, bounds=bounds, method="trust-constr")
        optimal_offset = res.x
    except:
        optimal_offset = initial_params
    
    tx_ant_gain_shifted = processor.tx_ant.getGain(df.aod_theta+optimal_offset[0], df.aod_phi)
    rx_ant_gain_shifted = processor.rx_ant.getGain(df.aoa_theta+optimal_offset[1]+df.yaw, df.aoa_phi-.07+df.pitch)
    
    pl_2ray_shifted = -20*np.log10(lmbd * np.abs(los_coef + ref_coef)/(4*np.pi)) - (tx_ant_gain_shifted + rx_ant_gain_shifted) 
    pl_free_shifted = 20*np.log10((4*np.pi*dist)/(lmbd)) - (tx_ant_gain_shifted + rx_ant_gain_shifted) 
    
    std_dev_shadowing = 0.8 
    
    pl_2ray_shifted = pl_2ray_shifted + np.random.normal(0, std_dev_shadowing, len(pl_2ray_shifted))
    pl_free_shifted = pl_free_shifted + np.random.normal(0, std_dev_shadowing, len(pl_free_shifted))
    
    new_model = pl_2ray + np.random.lognormal(0, 1e-10, len(pl_2ray))

    df["avg_pl_rolling"] = df["avg_pl"].rolling(window=10).mean()

    alpha = 0.2
    df["avg_pl_ewma"] = df["avg_pl"].ewm(alpha=alpha, adjust=True).mean()
    
    date = res_dir.split("/")[-2]
    df.to_csv(RES_DATE_DIR + date + "-results.csv", index=False)
    df.to_numpy().dump(RES_DATE_DIR + date + "-results.npy")
    
    pl_RMA_LoS  = [RMa3GPPChannelModel.pathLoss(row.h_dist, row.dist, freq/1e3, h_ue=row.alt) for i, row in df.iterrows()] - (tx_ant_gain + rx_ant_gain) 
    
    return pl_RMA_LoS, pl_2ray, pl_free, new_model

In [ ]:
data = { 
    "Center Frequency (MHz)": [],
    "Altitude (m)": [],
    "Speed": [],
    "Measured Mean PL (dB)": [],
    "Measured STD PL (dB)": [],
    "3GPP RMa Mean PL (dB)": [],
    "3GPP RMa STD PL (dB)": [],
    "2-Ray Mean PL (dB)": [],
    "2-Ray STD PL (dB)": [],
    "Free Space Mean PL (dB)": [],
    "Free Space STD PL (dB)": [],
}

for i in range(len(processed["meas"])):
    rma_loss, pl_2ray, pl_free,new_model = calc_pathloss(processed["meas"][i], processed["config"][i], processed["resultDir"][i])
    freq = processed["config"][i].USRP_CONF.CENTER_FREQ / 1e6
    print(f"Center Freq: {freq} MHz | Altitude: {processed['meas'][i]['alt'].max()} m | Path Loss Mean: {processed['meas'][i]['avg_pl'].mean()} dB | Path Loss STD: {processed['meas'][i]['avg_pl'].std()} dB ")
    print(f"3GPP RMA Path Loss Mean: {np.nanmean(rma_loss)} dB | 3GPP RMA Path Loss STD: {np.nanstd(rma_loss)} dB ")
    print(f"2-Ray Path Loss Mean: {np.nanmean(pl_2ray)} dB | 2-Ray Path Loss STD: {np.nanstd(pl_2ray)} dB ")
    print(f"Free Space Path Loss Mean: {np.nanmean(pl_free)} dB | Free Space Path Loss STD: {np.nanstd(pl_free)} dB ")
    
    data["Center Frequency (MHz)"].append(freq)
    data["Altitude (m)"].append(processed["meas"][i]["alt"].max())
    data["Speed"].append(processed["meas"][i]["speed"].mean())
    
    data["Measured Mean PL (dB)"].append(processed["meas"][i]["avg_pl"].mean())
    data["Measured STD PL (dB)"].append(processed["meas"][i]["avg_pl"].std())
    data["3GPP RMa Mean PL (dB)"].append(np.nanmean(rma_loss))
    data["3GPP RMa STD PL (dB)"].append(np.nanstd(rma_loss))
    data["2-Ray Mean PL (dB)"].append(np.nanmean(pl_2ray))
    data["2-Ray STD PL (dB)"].append(np.nanstd(pl_2ray))
    data["Free Space Mean PL (dB)"].append(np.nanmean(pl_free))
    data["Free Space STD PL (dB)"].append(np.nanstd(pl_free))

In [ ]:
df_pl = pd.DataFrame(data)
# Remove rows that include NaN values
# df_pl = df_pl.dropna()
df_pl.sort_values("Altitude (m)", inplace=True)

In [ ]:
# Line plot for Path Loss Mean vs. Altitude for each model
fig = go.Figure() 

colors = ['blue', 'red', 'green', 'purple']  # Base colors for models
line_styles = ['solid', 'dot', 'dash']

models = ["Measured", "3GPP RMa", "2-Ray", "Free Space"]
for model, color in zip(models, colors):
    for i, freq in enumerate(sorted(df_pl["Center Frequency (MHz)"].unique())):
        trace_df = df_pl[df_pl["Center Frequency (MHz)"] == freq]
        fig.add_trace(
            go.Scatter(
                x=trace_df["Altitude (m)"],
                y=trace_df[f"{model} STD PL (dB)"],
                mode='lines+markers',
                name=f"{model} at {freq} MHz",
                line=dict(color=color, dash=line_styles[i]),
                legendgroup=model
            )
        )

fig.update_layout(
    autosize=False,
    width=1920,
    height=1500,
    title_x=0.5,
    font=dict(family="Courier New, monospace", size=30),
    xaxis_title="Altitude (m)",
    yaxis_title="Path Loss σ<sup>2</sup>",
    legend=dict(
        x=0.01,  # Adjust legend position
        y=1.15,  # Above the plot
        orientation='h',  # Horizontal legend
        font=dict(size=30),  # Smaller font size for legend items
        traceorder='grouped'  # Group legend entries
    ),
    plot_bgcolor='rgba(230, 236, 245, 0.5)'
)

fig.show()

In [ ]:
def rma_pl_3GPP(f_mhz, d, h_bs, h_ue, h=5, w=10):
    """
    Calculate the RMa path loss using the 3GPP model for rural macro cells.
    
    Parameters:
    f (float): frequency in MHz
    d (float): distance between base station and user equipment in km
    h_bs (float): height of the base station in meters
    h_ue (float): height of the user equipment in meters
    h: avg. height of the building in meters
    w: avg. width of the street in meters
    
    Returns:
    float: path loss in dB
    """

    # Frequency factor
    c = constants.speed_of_light
    
    # Antenna heights
    h_bs_eff = max(35, h_bs)  # Effective base station height
    h_ue_eff = max(1.5, h_ue)  # Effective user equipment height
    
    f = f_mhz * 1e3
    
    # Distance factor
    d_bp = 2 * np.pi * h_bs_eff * h_ue_eff * f_mhz / c / 1e3  # Breakpoint distance in km
    
    # Path loss calculation
    if 10 < d < d_bp:
        # Free space path loss
        pl1 = 20 * np.log10(40*np.pi*d*f/3) + np.min(0.03*h, 10) * np.log10(d) - np.min(0.044*h, 14.77) + 0.002*np.log10(h)*d  
    elif d < 10:
        pl = np.nan
    else:
        # Two-ray ground reflection model (assuming the general rural environment)
        pl = 40 * np.log10(d) + 13.47 + 20 * np.log10(h_bs_eff * h_ue_eff) - 3.2 * (np.log10(11.75 * h_ue_eff))**2 - 4.97

    return pl


In [ ]:
fig = go.Figure()
fig.add_trace(go.Scatter(x=df["time"], y=df["aod_phi"], name="AOD Elevation"))
fig.add_trace(go.Scatter(x=df["time"], y=df["aoa_phi"], name="AOA Elevation"))

fig.add_trace(go.Scatter(x=df["time"], y=df["aod_theta"], name="AOD Azimuth"))
fig.add_trace(go.Scatter(x=df["time"], y=df["aoa_theta"], name="AOA Azimuth"))

fig.update_layout(
    autosize=False,
    width=1920,
    height=1080,
    title_text='AOD/AOA vs Time',
    title_x=0.5,
    font=dict(family="Courier New, monospace", size=18),
    xaxis_title="Time (s)",
    yaxis_title="Angle (radians)",
    legend=dict(
        yanchor="top",
        y=0.99,
        xanchor="left",
        x=0.01
    )
)

fig.show()

In [ ]:
def calculate_endpoint(lat, lon, azimuth, distance=0.1):
    delta_lat = distance * np.cos(azimuth)
    delta_lon = distance * np.sin(azimuth)
    
    return lat + delta_lat, lon + delta_lon


In [ ]:
df["end_points"] = df.apply(lambda x: calculate_endpoint(x["lat"], x["lon"], x["aod_theta"],1e-4), axis=1)
df["end_lon"] = df["end_points"].apply(lambda x: x[1])
df["end_lat"] = df["end_points"].apply(lambda x: x[0])

In [ ]:
fig, ax = plt.subplots()

for i, _ in df.iterrows():
    if i  % 8 == 0:
        ax.arrow(df.lon[i], df.lat[i], df.end_lon[i] - df.lon[i], df.end_lat[i] - df.lat[i], width=1e-7, head_width=2e-5, head_length=3e-5, fc='red', ec='red')

ax.annotate('Source', xy=(LW1[1], LW1[0]), xytext=(LW1[1]+3e-5, LW1[0]+3e-5))

ax.set_xlabel("Longitude")
ax.set_ylabel("Latitude")
ax.set_title("AOD Azimuth")


ax.plot(LW1[1], LW1[0], marker='o', markersize=10, color='red')

fig.set_size_inches(20, 11)


fig.show()

fig.savefig(RESULT_DIR+"AOD_Azimuth.png")


In [ ]:
px.set_mapbox_access_token(
    "pk.eyJ1Ijoic2VnbWVudGZhdWx0IiwiYSI6ImNsYTh5bTU5aDA2OWQzb3F4c29rajR4MjAifQ.XyOdrlhpVOUJZ18YFNwd8A"
)

fig = px.scatter_mapbox(
    lat=df.lat,
    lon=df.lon,
    color=df.avgPower,
    labels={"avgPower": "Power(dBm)"},
    title=f"Power vs Location for Center Freq. = {freq} MHz and {waveType} waveform",
    mapbox_style="satellite",
    color_continuous_scale=px.colors.sequential.thermal,
    size_max=11,
    size=df["lat"],
    zoom=16.2,
    width=1400,
    height=1000
)

for i, _ in df.iterrows():
    if i % 10 == 0:
        fig.add_trace(
            go.Scattermapbox(
                mode="markers+lines",
                lon=[df.lon[i], df.end_lon[i]],
                lat=[df.lat[i], df.end_lat[i]],
                marker=dict(
                    symbol="arrow-up",
                    size=20
                ),
                line={"width": 3},
            )
        )
        
        
fig.add_trace(go.Scattermapbox(
    mode="markers",
    lon=[LW1[1]],
    lat=[LW1[0]],
    marker=dict(
        size=20,
        color="red"
    ),
    name="Source"
))

fig.add_annotation(
    go.layout.Annotation(
        text="Source",
        x=LW1[1]+1e-5,
        y=LW1[0]+1e-5,
        showarrow=False
    )
)

fig.update_layout(showlegend=False)

fig.show()

In [ ]:
plt.plot(df["time"], np.rad2deg(df["pitch"]))
plt.plot(df["time"], np.rad2deg(df["roll"]))
plt.plot(df["time"], np.rad2deg(df["yaw"]))


plt.legend(["Pitch", "Roll", "Yaw"])
plt.xlabel("Time (s)")
plt.ylabel("Angle (degrees)")

plt.savefig(RESULT_DIR+"Angles.png")

In [ ]:
lambda_ = 0.3  # Wavelength in meters (corresponds to 3.5 GHz frequency)
ht100 = 100  # Transmitter height for the first scenario in meters
ht30 = 30    # Transmitter height for the second scenario in meters
ht2 = 2      # Transmitter height for the third scenario in meters
hr = 2       # Receiver height in meters

# Initialize lists for storing results
distances = []
p100 = []
p30 = []
p2 = []
pfsl = []

# Calculate path loss for different distances
for i in range(30, 451):
    d = 10 ** (i / 1000)
    distances.append(d)
    fspower = (lambda_ / (4 * np.pi * d)) ** 2
    power100 = fspower * 4 * (np.sin(2 * np.pi * hr * ht100 / (lambda_ * d))) ** 2
    power30 = fspower * 4 * (np.sin(2 * np.pi * hr * ht30 / (lambda_ * d))) ** 2
    power2 = fspower * 4 * (np.sin(2 * np.pi * hr * ht2 / (lambda_ * d))) ** 2
    p100.append(20 * np.log10(power100))
    p30.append(20 * np.log10(power30))
    p2.append(20 * np.log10(power2))
    pfsl.append(20 * np.log10(fspower))

# Plotting
plt.figure(figsize=(10, 6))
plt.semilogx(distances, p100, 'g-', label='ht=100m')
plt.semilogx(distances, p30, 'b-', label='ht=30m')
plt.semilogx(distances, p2, 'r-', label='ht=2m')
plt.semilogx(distances, pfsl, 'y-', label='Free Space')
plt.xlabel('Distance (m)')
plt.ylabel('Path Loss (dB)')
plt.title('Two-Ray Ground Reflection Model Path Loss')
plt.legend()
plt.grid(True)
plt.show()

# CIR 

In [ ]:
df = processed["meas"][0]
cir = df["corr"].copy()

max_delay = []
x_line = []
y_line = []
for i in range(len(cir)):
    # _st = df["orig_peaks"][i][1] - _cir_off - 100
    cr =  df["corr"][i].astype(float)
    cr = np.array(10*np.log10(np.abs(cr)+1e-8))
    # cr = np.array(np.abs(cr))
    # cr = np.roll(cr, _st)
    x_line.append(df["dist"][i]/250)
    y_line.append(df["time"][i])
    cr = cr[560:700]
    threshold = 0.3 * np.max(np.abs(cr))

    # Identify significant paths based on the threshold
    significant_indices = np.where(np.abs(cr) > threshold)[0]
    time_indices = np.linspace(0, len(cr), len(cr)) * (1/cfg.USRP_CONF.SAMPLE_RATE) * 1e6
    # Calculate max excess delay
    if significant_indices.size > 0:
        max_excess_delay = time_indices[significant_indices[-1]] - time_indices[significant_indices[0]]
    else:
        max_excess_delay = 0
    
    max_delay.append(max_excess_delay)
    # cr -= cr.mean()
    cir[i] = cr

z_line = np.zeros_like(x_line) + 23

print(f"Average Max Delay: {np.mean(max_delay)} (μs)")
x_ax = np.linspace(0, len(cr), len(cr)) * (1/cfg.USRP_CONF.SAMPLE_RATE) * 1e6

line = go.Scatter3d(
    x=x_line,
    y=y_line,
    z=z_line,
    mode='lines',
    line=dict(color='black', width=8)
)

fig = go.Figure(data=[go.Surface(z=cir,y=df["time"],x=x_ax, colorscale='Turbo', 
                      colorbar=dict(title=dict(text='Magnitude<br>(dBm)', font=dict(size=16)))), line])
                      
fig.update_layout(autosize=False,
                  width=1000, height=1000)

# change font size of the axis titles 
fig.update_layout(scene = dict(
                    xaxis_title='Sample Time (μs)',
                    yaxis_title='Experiment Time (s)',
                    zaxis_title='Magnitude (dBm)'),
                    font=dict(size=16),
                    margin=dict(r=10, b=10, l=10, t=10),
)



fig.update_layout(scene_camera=dict(
    up=dict(x=0, y=0, z=2),
    center=dict(x=0, y=0, z=0),
    eye=dict(x=0.6, y=1.6, z=1.35)
))


fig.add_annotation(
    dict(
        x=0.25,
        y=0.94,
        textangle=-7.5,
        text="GPS Distance over time (m)",
        showarrow=False,
        xanchor="left",
        xref="paper",
        yref="paper",
        font=dict(size=16)
    )
)

fig.add_annotation(
    dict(
        x=0.64,
        y=0.93,
        textangle=-6,
        text="0",
        showarrow=False,
        xanchor="left",
        xref="paper",
        yref="paper",
        font=dict(size=16)
    )
)

fig.add_annotation(
    dict(
        x=0.52,
        y=0.92,
        textangle=-6,
        text="130",
        showarrow=False,
        xanchor="left",
        xref="paper",
        yref="paper",
        font=dict(size=16)
    )
)

fig.add_annotation(
    dict(
        x=0.42,
        y=0.91,
        textangle=-6,
        text="260",
        showarrow=False,
        xanchor="left",
        xref="paper",
        yref="paper",
        font=dict(size=16)
    )
)

fig.add_annotation(
    dict(
        x=0.31,
        y=0.895,
        textangle=-6,
        text="390",
        showarrow=False,
        xanchor="left",
        xref="paper",
        yref="paper",
        font=dict(size=16)
    )
)

fig.add_annotation(
    dict(
        x=0.19,
        y=0.883,
        textangle=-6,
        text="520",
        showarrow=False,
        xanchor="left",
        xref="paper",
        yref="paper",
        font=dict(size=16)
    )
)


fig.show()
fig.write_html( RESULT_DIR+"CIRvsTime3D.html")